task name: Resolve Gam Task

task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML.FastTree"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext();

In [ ]:
private class Data
{
    public bool Label { get; set; }

    [VectorType(2)]
    public float[] Features { get; set; }
}

private static IEnumerable<Data> GenerateData(int numExamples = 25000, int seed = 1)
{
    var rng = new Random(seed);
    float centeredFloat() => (float)(rng.NextDouble() - 0.5);
    for (int i = 0; i < numExamples; i++)
    {
        var data = new Data
        {
            Features = new float[2] { centeredFloat(), centeredFloat() }
        };
        data.Label = Sigmoid(Parabola(data.Features[0]) + SimplePiecewise(data.Features[1]) + centeredFloat()) > 0.5;

        yield return data;
    }
}

private static float Parabola(float x) => x * x;

private static float SimplePiecewise(float x)
{
    if (x < 0)
        return 0;
    else if (x < 0.25)
        return 1;
    else
        return 0;
}

private static double Sigmoid(double x) => 1.0 / (1.0 + Math.Exp(-1 * x));

var samples = GenerateData();
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var dataSets = mlContext.Data.TrainTestSplit(data);
var trainSet = dataSets.TrainSet;
var validSet = dataSets.TestSet;

In [ ]:
var trainer = mlContext.BinaryClassification.Trainers.Gam(maximumBinCountPerFeature: 16);
var model = trainer.Fit(trainSet, validSet);

In [ ]:
var gam = model.Model.SubModel;

In [ ]:
Console.WriteLine($"Average prediction: {gam.Bias:0.00}");
for (int i = 0; i < gam.NumberOfShapeFunctions; i++)
{
    Console.WriteLine();
    var binUpperBounds = gam.GetBinUpperBounds(i);
    var binEffects = gam.GetBinEffects(i);
    Console.WriteLine($"Feature{i}");
    for (int j = 0; j < binUpperBounds.Count; j++)
        Console.WriteLine($"x < {binUpperBounds[j]:0.00} => {binEffects[j]:0.000}");
}